# Quickstart for Carsus

In this guide you will learn how to create atomic data files suitable for [TARDIS](https://github.com/tardis-sn/tardis).

## Atomic Weights and Ionization Energies

Download atomic weights and ionization energies from the National Institute of Standards and Technology (NIST).


<div class="alert alert-info">

**Read first:**

Get familiar with [Notation in Carsus](development/notation.rst) and learn how to select different sets of ions.
    
</div>

In [1]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies

 ChiantiPy version 0.8.4 
 found PyQt5 widgets
 using PyQt5 widgets


In [2]:
atomic_weights = NISTWeightsComp()
ionization_energies = NISTIonizationEnergies('H-Zn')

[ carsus.io.nist.weightscomp][   INFO] - Downloading data from the NIST Atomic Weights and Isotopic Compositions Database. (weightscomp.py:49)


[  carsus.io.nist.ionization][   INFO] - Downloading ionization energies from the NIST Atomic Spectra Database. (ionization.py:65)


## Levels, Lines, Collisions and Cross-sections

Carsus supports three sources of energy levels and spectral lines: GFALL, CHIANTI and CMFGEN.

### GFALL

The Robert Kurucz's Atomic Linelist (GFALL) reader is the main source of **levels and lines**.

In [3]:
!wget -qO /tmp/gfall.dat https://media.githubusercontent.com/media/tardis-sn/carsus-db/master/gfall/gfall_latest.dat

In [4]:
from carsus.io.kurucz import GFALLReader

gfall_reader = GFALLReader('H-Zn', '/tmp/gfall.dat')

[     carsus.io.kurucz.gfall][WARNING] - A specific combination to identify unique levels from GFALL data has not been given. Defaulting to ["energy", "j"]. (gfall.py:95)


<div class="alert alert-warning">

**Warning:**
    
Creating a `GFALLReader` instance is **required**. Other sources of levels and lines are **optional**.

</div>

### CHIANTI

The Chianti Atomic Database reader provides levels and lines but also **collision strengths**.

In [5]:
from carsus.io.chianti_ import ChiantiReader

chianti_reader = ChiantiReader('H-He', collisions=True, priority=20)

By default `priority` parameter is set to `10`. Increase this value if you want to keep CHIANTI levels and lines over GFALL.

### CMFGEN

The atomic database of [CMFGEN](http://kookaburra.phyast.pitt.edu/hillier/web/CMFGEN.htm) is a source of levels, lines and (optionally) ionization energies and **photoionization cross-sections**.

In [6]:
from carsus.io.cmfgen import CMFGENReader

cmfgen_reader = CMFGENReader.from_config('Si 0; Si 1', '/tmp/atomic', priority=30, 
                                         ionization_energies=True, cross_sections=True)

[      carsus.io.cmfgen.base][WARNING] - Selecting H 0 required to ingest cross-sections. (base.py:492)


[      carsus.io.cmfgen.base][   INFO] - Configuration schema found for H 0. (base.py:504)


[      carsus.io.cmfgen.base][   INFO] - Configuration schema found for Si 0. (base.py:504)


[      carsus.io.cmfgen.base][   INFO] - Configuration schema found for Si 1. (base.py:504)


[      carsus.io.cmfgen.base][   INFO] - Loading atomic data for H 0. (base.py:712)


[      carsus.io.cmfgen.base][   INFO] - Loading atomic data for Si 0. (base.py:712)


[      carsus.io.cmfgen.base][   INFO] - Loading atomic data for Si 1. (base.py:712)


<div class="alert alert-warning">

**Warning:**
    
Remember that cross-sections requires data from `H 0` so give this the reader enough `priority`. 

</div>

If you want to use the **ionization energies** from CMFGEN, you will need to apply this dirty hack to merge both sources:

In [7]:
import copy
import pandas as pd

combined_ionization_energies = copy.deepcopy(ionization_energies)
combined_ionization_energies.base = cmfgen_reader.ionization_energies.combine_first(ionization_energies.base)

## Zeta Data

Knox S. Long's ground state recombinations fractions ($\zeta$).

In [8]:
from carsus.io.zeta import KnoxLongZeta

zeta_data = KnoxLongZeta()

## Dump to HDF5

Finally, create a `TARDISAtomData` object and dump the data with the `to_hdf` method.

In [9]:
from carsus.io.output import TARDISAtomData

atom_data = TARDISAtomData(atomic_weights,
                           combined_ionization_energies,
                           gfall_reader,
                           zeta_data,
                           chianti_reader,
                           cmfgen_reader)

[      carsus.io.output.base][   INFO] - Ingesting energy levels. (base.py:299)


[     carsus.io.kurucz.gfall][   INFO] - Parsing GFALL from: /tmp/gfall.dat (gfall.py:148)


[      carsus.io.output.base][   INFO] - GFALL selected species: Li 0, Li 1, Be 0, Be 1, Be 2, B 0, B 1, B 2, B 3, C 0, C 1, C 2, C 3, N 0, N 1, N 2, N 3, N 4, N 5, O 0, O 1, O 2, O 3, O 4, O 5, F 0, F 1, F 2, F 3, F 4, F 5, Ne 0, Ne 1, Ne 2, Ne 3, Ne 4, Ne 5, Na 0, Na 1, Na 2, Na 3, Na 4, Na 5, Mg 0, Mg 1, Mg 2, Mg 3, Mg 4, Mg 5, Al 0, Al 1, Al 2, Al 3, Al 4, Al 5, Si 2, Si 3, Si 4, Si 5, P 0, P 1, P 2, P 3, P 4, P 5, S 0, S 1, S 2, S 3, S 4, S 5, Cl 0, Cl 1, Cl 2, Cl 3, Cl 4, Ar 0, Ar 1, Ar 2, Ar 3, Ar 4, K 0, K 1, K 2, K 3, K 4, Ca 0, Ca 1, Ca 2, Ca 3, Ca 4, Ca 5, Ca 6, Ca 7, Ca 8, Sc 0, Sc 1, Sc 2, Sc 3, Sc 4, Sc 5, Sc 6, Sc 7, Sc 8, Ti 0, Ti 1, Ti 2, Ti 3, Ti 4, Ti 5, Ti 6, Ti 7, Ti 8, V 0, V 1, V 2, V 3, V 4, V 5, V 6, V 7, V 8, Cr 0, Cr 1, Cr 2, Cr 3, Cr 4, Cr 5, Cr 6, Cr 7, Cr 8, Mn 0, Mn 1, Mn 2, Mn 3, Mn 4, Mn 5, Mn 6, Mn 7, Mn 8, Fe 0, Fe 1, Fe 2, Fe 3, Fe 4, Fe 5, Fe 6, Fe 7, Fe 8, Co 0, Co 1, Co 2, Co 3, Co 4, Co 5, Co 6, Co 7, Co 8, Ni 0, Ni 1, Ni 2, Ni 3, Ni 4, Ni 5, Ni 

[      carsus.io.output.base][   INFO] - Chianti selected species: He 0, He 1. (base.py:337)


[      carsus.io.output.base][   INFO] - CMFGEN selected species: H 0, Si 0, Si 1. (base.py:341)


...

[      carsus.io.output.base][   INFO] - Ingesting collisional strengths. (base.py:588)


[      carsus.io.output.base][   INFO] - Matching collisions and levels. (base.py:600)


[      carsus.io.output.base][   INFO] - Ingesting photoionization cross-sections. (base.py:668)


[      carsus.io.output.base][   INFO] - Matching levels and cross sections. (base.py:671)


[      carsus.io.output.base][   INFO] - Finished. (base.py:79)


In [10]:
atom_data.to_hdf('kurucz_cd23_chianti_He_cmfgen_H_Si_I-II.h5')

[      carsus.io.output.base][   INFO] - Signing TARDISAtomData. (base.py:1038)


[      carsus.io.output.base][   INFO] - MD5: ab39a1308623156cc5752ed31e133b21 (base.py:1039)


[      carsus.io.output.base][   INFO] - UUID1: f9b1520e7d3311ecb6ee000d3a1ca7ca (base.py:1040)


You are done! Now you can use your file to run TARDIS simulations.

## Metadata

Carsus stores metadata inside the HDF5 files to ensure reproducibility. This metadata includes a checksum per `DataFrame`, the current version of every dataset and relevant software versions. 

In [11]:
import pandas as pd

In [12]:
pd.read_hdf('kurucz_cd23_chianti_He_cmfgen_H_Si_I-II.h5', key='meta')

value
field    key                                              
md5sum   atom_data                    634785f05c4868f929c8
         collision_data               0f19bec97c3b770c6ed7
         collision_data_temperatures  1b7075429ee24b944ed5
         ionization_data              8ccad0f17651040d2dcb
         levels                       ea5cbc1cd622e5c7e2bd
         lines                        9f71e0663f07b997d533
         macro_atom_data              1d55b4ff7204aae856e8
         macro_atom_references        5ed4563fc49eca988451
         photoionization_data         ac3b5b5088d7c5af9265
         zeta_data                    f2496cec5c219dfb6d63
datasets nist_weights                                  4.1
         nist_spectra                                  5.9
         gfall.dat                    2704fbda0b8cba61bb70
         chianti_data                                9.0.1
software python                                     3.6.15
         carsus                        0.1.dev661+ga194067
         astropy                                     3.2.3
         numpy                                      1.15.4
         pandas                                      1.0.5
         pyarrow                                    0.14.1
         tables                                      3.6.1
         ChiantiPy                                   0.8.4

In [13]:
# nbsphinx hidden cell
!rm kurucz_cd23_chianti_He_cmfgen_H_Si_I-II.h5